In [1]:
from pathlib import Path
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [5]:
from pathlib import Path
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

# First load all sequences as before
all_records = []

for path in Path('./outputs').rglob('*.fa'):
    file_info = {
        "filepath": str(path),
        "folder": path.parts[1],
        "filename": path.parts[3]
    }
    
    with open(path) as handle:
        sequences = list(SeqIO.parse(handle, "fasta"))
        
        for seq_record in sequences[1:]:
            desc_parts = dict(item.split('=') for item in seq_record.description.split(', ') 
                            if '=' in item)
            
            # Clean the sequence: remove any whitespace and join split lines
            clean_sequence = str(seq_record.seq).replace('\n', '').replace(' ', '')
            
            sequence_data = {
                **file_info,
                "sequence": clean_sequence,
                "temperature": desc_parts.get('T', ''),
                "sample": desc_parts.get('sample', ''),
                "score": float(desc_parts.get('score', 0)),
                "global_score": float(desc_parts.get('global_score', 0)),
                "seq_recovery": float(desc_parts.get('seq_recovery', 0))
            }
            all_records.append(sequence_data)

# Create DataFrame and process unique sequences
df = pd.DataFrame(all_records)

# Get unique sequences with their best (lowest) scores
best_sequences = (df.sort_values('score')
                   .drop_duplicates('sequence', keep='first'))

# Sort by score
sorted_sequences = best_sequences.sort_values('score')

# Create output FASTA file
output_filename = "unique_sequences_sorted.fa"

with open(output_filename, 'w') as output_handle:
    for idx, row in sorted_sequences.iterrows():
        description = (f"score={row['score']:.4f}, "
                      f"global_score={row['global_score']:.4f}, "
                      f"seq_recovery={row['seq_recovery']:.4f}, "
                      f"source_file={row['filename']}")
        
        # Create SeqRecord object
        record = SeqRecord(
            Seq(row['sequence']),
            id=f"sequence_{idx+1}",
            description=description
        )
        
        # Write the record, forcing single line sequences
        output_handle.write(f">{record.description}\n{str(record.seq)}\n")

# Print summary and some statistics about sequence lengths
print(f"Total input sequences: {len(df)}")
print(f"Unique sequences: {len(sorted_sequences)}")
print(f"Written to: {output_filename}")
print("\nSequence length statistics:")
print(sorted_sequences['sequence'].str.len().describe())
print("\nFirst few sequences by score:")
print(sorted_sequences[['sequence', 'score']].head())

# Optional: Check for any unusually long sequences
long_sequences = sorted_sequences[sorted_sequences['sequence'].str.len() > 100]
if not long_sequences.empty:
    print("\nUnusually long sequences found:")
    print(long_sequences[['sequence', 'score', 'filename']])

Total input sequences: 46080
Unique sequences: 34603
Written to: unique_sequences_sorted.fa

Sequence length statistics:
count    34603.000000
mean        46.106089
std         16.443948
min         21.000000
25%         34.000000
50%         41.000000
75%         61.000000
max         80.000000
Name: sequence, dtype: float64

First few sequences by score:
                                                sequence   score
27462  EELEKLKKELEEMIKRAEKAAEDGTGLDSIDGILTKVKNKLEEAKK...  0.9873
30871  LEEVKKKLEELKEKFKKEGVISREEAVEAYALVEYLKEKGELTTEM...  0.9874
15519  IEKVKEELKELMKKYKEEGVISREDAVRAYALVDYLKEKGLITTEM...  1.0003
29788  GAHLAAIRAALASFETAWQAARTLLALARAGDPEVRALLAELGLLP...  1.0007
25759  IEKVKEELKELMKKYKEEKVISREDAVRAYALVDYLKEKGLITTEM...  1.0021


In [3]:
all_records = []

for path in Path('./outputs').rglob('*.fa'):
    file_info = {
        "filepath": str(path),
        "folder": path.parts[1],
        "filename": path.parts[3]
    }
    
    with open(path) as handle:
        sequences = list(SeqIO.parse(handle, "fasta"))
        
        for seq_record in sequences[1:]:
            desc_parts = dict(item.split('=') for item in seq_record.description.split(', ') 
                            if '=' in item)
            
            sequence_data = {
                **file_info,  # Unpack file info
                "sequence": str(seq_record.seq),
                "temperature": desc_parts.get('T', ''),
                "sample": desc_parts.get('sample', ''),
                "score": float(desc_parts.get('score', 0)),
                "global_score": float(desc_parts.get('global_score', 0)),
                "seq_recovery": float(desc_parts.get('seq_recovery', 0))
            }
            all_records.append(sequence_data)

df = pd.DataFrame(all_records)
print(df.head())

                                            filepath                  folder  \
0  outputs/insol_st0.3_bn0.01_s42/seqs/cd20ecm_hs...  insol_st0.3_bn0.01_s42   
1  outputs/insol_st0.3_bn0.01_s42/seqs/cd20ecm_hs...  insol_st0.3_bn0.01_s42   
2  outputs/insol_st0.3_bn0.01_s42/seqs/cd20ecm_hs...  insol_st0.3_bn0.01_s42   
3  outputs/insol_st0.3_bn0.01_s42/seqs/cd20ecm_hs...  insol_st0.3_bn0.01_s42   
4  outputs/insol_st0.3_bn0.01_s42/seqs/cd20ecm_hs...  insol_st0.3_bn0.01_s42   

               filename                                           sequence  \
0  cd20ecm_hsb_05_24.fa  SLADKIVEALKNAAKMTQEALAQSRALIAKNSDLLDEYNKILKITL...   
1  cd20ecm_hsb_05_24.fa  SLTDLILNYLKEAGKMTDSALKTSQQLLSKDPDALAAYNEILKLVK...   
2  cd20ecm_hsb_05_24.fa  SLSDIIVEKLLKAREMNDSALKTSEAQISKNKEAKKEFEKIKEITL...   
3  cd20ecm_hsb_05_24.fa  SLSDLVLEALKEAGKMNSAALKTSKSLIESNKDLLEEFNKILDLVK...   
4  cd20ecm_hsb_05_24.fa  SVTDEILESLKEAGKMSKEALEKSRNLIKGDPQALKDFEEIKKIVE...   

  temperature sample   score  global_score  seq_re

In [4]:
best_sequences = (df.sort_values('score')  # Sort by score
                   .drop_duplicates('sequence', keep='first'))  # Keep first (best) occurrence
sorted_sequences = best_sequences.sort_values('score')

# Create output FASTA file
output_filename = "unique_sequences_sorted.fa"

with open(output_filename, 'w') as output_handle:
    for idx, row in sorted_sequences.iterrows():
        # Create description string with relevant information
        description = (f"score={row['score']:.4f}, "
                      f"global_score={row['global_score']:.4f}, "
                      f"seq_recovery={row['seq_recovery']:.4f}, "
                      f"source_file={row['filename']}")
        
        # Create SeqRecord object
        record = SeqRecord(
            Seq(row['sequence']),
            id=f"sequence_{idx+1}",
            description=description
        )
        
        # Write the record
        SeqIO.write(record, output_handle, "fasta")

# Print summary
print(f"Total input sequences: {len(df)}")
print(f"Unique sequences: {len(sorted_sequences)}")
print(f"Written to: {output_filename}")
print("\nFirst few sequences by score:")
print(sorted_sequences[['sequence', 'score']].head())

Total input sequences: 46080
Unique sequences: 34603
Written to: unique_sequences_sorted.fa

First few sequences by score:
                                                sequence   score
27462  EELEKLKKELEEMIKRAEKAAEDGTGLDSIDGILTKVKNKLEEAKK...  0.9873
30871  LEEVKKKLEELKEKFKKEGVISREEAVEAYALVEYLKEKGELTTEM...  0.9874
15519  IEKVKEELKELMKKYKEEGVISREDAVRAYALVDYLKEKGLITTEM...  1.0003
29788  GAHLAAIRAALASFETAWQAARTLLALARAGDPEVRALLAELGLLP...  1.0007
25759  IEKVKEELKELMKKYKEEKVISREDAVRAYALVDYLKEKGLITTEM...  1.0021
